In [143]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Deliverable 1: Preprocessing the Data for PCA

In [144]:
# Load the crypto_data.csv dataset.
file_path = "/Users/iralee/Cryptocurrencies/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [145]:
# Keep all the cryptocurrencies that are being traded.
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
crypto_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [146]:
# Drop the IsTrading column
# Remove columns
crypto_df = crypto_df.drop(['IsTrading','Unnamed: 0'], axis=1)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
1243,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,SHA-256,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [147]:
# Keep all the cryptocurrencies that have a working algorithm.
# find null values
for column in crypto_df.columns:
    print(f"Column{column} has {crypto_df[column].isnull().sum()}null values")

ColumnCoinName has 0null values
ColumnAlgorithm has 0null values
ColumnProofType has 0null values
ColumnTotalCoinsMined has 459null values
ColumnTotalCoinSupply has 0null values


In [148]:
# Remove rows that have at least one null value
# Drop null rows
crypto_df = crypto_df.dropna()
for column in crypto_df.columns:
    print(f"Column{column} has {crypto_df[column].isnull().sum()}null values")
crypto_df

ColumnCoinName has 0null values
ColumnAlgorithm has 0null values
ColumnProofType has 0null values
ColumnTotalCoinsMined has 0null values
ColumnTotalCoinSupply has 0null values


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [149]:
# Filter the crypto_df DataFrame so it only has rows where coins have been mined
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [150]:
# Create a new DataFrame that holds only the cryptocurrency names, and use the crypto_df DataFrame index as the index for this new DataFrame
# Create a new DataFrame that holds only the cryptocurrencies names.
## drop the class
cryptoname_df = crypto_df.drop(['Algorithm','ProofType','TotalCoinsMined','TotalCoinSupply'],axis=1)
cryptoname_df.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [151]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(['CoinName'], axis=1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [152]:
# Use get_dummies() to create variables for text features.
crypto = pd.get_dummies(crypto_df['Algorithm'])
dummy = pd.get_dummies(crypto_df['ProofType'])
combined = pd.concat([crypto, dummy], axis=1)
df = crypto_df.merge(combined, left_index= True,right_index = True)
df = df.drop(['Algorithm', 'ProofType'], axis = 1)
df

,TotalCoinsMined,TotalCoinSupply,1GB AES Pattern Search,536,Argon2d,BLAKE256,Blake,Blake2S,Blake2b,C11,...,PoW/PoS,PoW/PoS,PoW/PoW,PoW/nPoS,Pos,Proof of Authority,Proof of Trust,TPoS,Zero-Knowledge Proof,dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1242,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1245,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
# Standardize the data with StandardScaler().
df_scaled = StandardScaler().fit_transform(df)
print(df_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


## Deliverable 2: Reducing Data Dimensions Using PCA

In [154]:
# apply PCA to reduce the dimensions to three principal components
# Initialize PCA model
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)
df_pca

array([[-0.34935156,  1.02419829, -0.57602811],
       [-0.33270038,  1.02436271, -0.57646794],
       [ 2.31026365,  1.62068054, -0.65358451],
       ...,
       [ 0.33227152, -2.30522397,  0.36432053],
       [-0.16289248, -2.01607231,  0.56824823],
       [-0.2966672 ,  0.86283728, -0.26307978]])

In [155]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=df_pca, columns=["principal component 1", "principal component 2", "principal component 3"], index = crypto_df.index)
pcs_df

,principal component 1,principal component 2,principal component 3
0,-0.349352,1.024198,-0.576028
2,-0.332700,1.024363,-0.576468
5,2.310264,1.620681,-0.653585
7,-0.135461,-1.297543,0.165404
8,-0.139417,-2.018727,0.356509
...,...,...,...
1238,2.491320,0.853402,-0.191558
1242,-0.347398,1.024083,-0.576060
1245,0.332272,-2.305224,0.364321
1246,-0.162892,-2.016072,0.568248


## Deliverable 3: Clustering Cryptocurrencies Using K-means

In [161]:
# Create an elbow curve to find the best value for K.
inertia = []
k=list(range(1,11))

# Calculate the inertia for the range of K Values 
for i in k: 
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve 
elbow_data = {"k" : k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [162]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [163]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = cryptoname_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.349352,1.024198,-0.576028,42 Coin,0
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.332700,1.024363,-0.576468,404Coin,0
5,X13,PoW/PoS,2.927942e+10,314159265359,2.310264,1.620681,-0.653585,EliteCoin,0
7,SHA-256,PoW,1.792718e+07,21000000,-0.135461,-1.297543,0.165404,Bitcoin,3
8,Ethash,PoW,1.076842e+08,0,-0.139417,-2.018727,0.356509,Ethereum,3
9,Scrypt,PoW,6.303924e+07,84000000,-0.177221,-1.117117,-0.008458,Litecoin,3
10,X11,PoW/PoS,9.031294e+06,22000000,-0.387766,1.203964,-0.497296,Dash,0
11,CryptoNight-V7,PoW,1.720114e+07,0,-0.124656,-2.250288,0.368145,Monero,3
12,Ethash,PoW,1.133597e+08,210000000,-0.137861,-2.018826,0.356485,Ethereum Classic,3
13,Equihash,PoW,7.383056e+06,21000000,-0.162891,-2.016072,0.568248,ZCash,3


In [165]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = cryptoname_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.349352,1.024198,-0.576028,42 Coin,0
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.332700,1.024363,-0.576468,404Coin,0
5,X13,PoW/PoS,2.927942e+10,314159265359,2.310264,1.620681,-0.653585,EliteCoin,0
7,SHA-256,PoW,1.792718e+07,21000000,-0.135461,-1.297543,0.165404,Bitcoin,3
8,Ethash,PoW,1.076842e+08,0,-0.139417,-2.018727,0.356509,Ethereum,3
9,Scrypt,PoW,6.303924e+07,84000000,-0.177221,-1.117117,-0.008458,Litecoin,3
10,X11,PoW/PoS,9.031294e+06,22000000,-0.387766,1.203964,-0.497296,Dash,0
11,CryptoNight-V7,PoW,1.720114e+07,0,-0.124656,-2.250288,0.368145,Monero,3
12,Ethash,PoW,1.133597e+08,210000000,-0.137861,-2.018826,0.356485,Ethereum Classic,3
13,Equihash,PoW,7.383056e+06,21000000,-0.162891,-2.016072,0.568248,ZCash,3


## Deliverable 4: Visualizing Cryptocurrencies Results

In [132]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    hover_name="CoinName",
    hover_data=["Algorithm"],
    x="principal component 1",
    y="principal component 2",
    z="principal component 3",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [168]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [169]:
# Print the total number of tradable cryptocurrencies in the clustered_df DataFrame
print(clustered_df["CoinName"].count)

<bound method Series.count of 0           42 Coin
2           404Coin
5         EliteCoin
7           Bitcoin
8          Ethereum
           ...     
1238         ZEPHYR
1242        Gapcoin
1245         Beldex
1246        Horizen
1247    BitcoinPlus
Name: CoinName, Length: 532, dtype: object>


In [170]:
# Use the MinMaxScaler().fit_transform method to scale the TotalCoinSupply and TotalCoinsMined columns between the given range of zero and one
scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(scaled)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [175]:
# Create a new DataFrame using the clustered_df DataFrame index that contains the scaled data you created in Step 5.
newclustered_df=pd.DataFrame(
    data=scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index = clustered_df.index)
newclustered_df

,TotalCoinSupply,TotalCoinsMined
0,4.200000e-11,0.000000e+00
2,5.320000e-04,1.065855e-03
5,3.141593e-01,2.957551e-02
7,2.100000e-05,1.810842e-05
8,0.000000e+00,1.087731e-04
...,...,...
1238,2.000000e-03,2.020225e-03
1242,2.500000e-04,1.508199e-05
1245,1.400223e-03,9.901351e-04
1246,2.100000e-05,7.370282e-06


In [176]:
# Add the CoinName column from the clustered_df DataFrame to the new DataFrame
newclustered_df=pd.concat([newclustered_df,clustered_df["CoinName"]], axis=1)
newclustered_df

,TotalCoinSupply,TotalCoinsMined,CoinName
0,4.200000e-11,0.000000e+00,42 Coin
2,5.320000e-04,1.065855e-03,404Coin
5,3.141593e-01,2.957551e-02,EliteCoin
7,2.100000e-05,1.810842e-05,Bitcoin
8,0.000000e+00,1.087731e-04,Ethereum
...,...,...,...
1238,2.000000e-03,2.020225e-03,ZEPHYR
1242,2.500000e-04,1.508199e-05,Gapcoin
1245,1.400223e-03,9.901351e-04,Beldex
1246,2.100000e-05,7.370282e-06,Horizen


In [177]:
# Add the Class column from the clustered_df DataFrame to the new DataFrame
newclustered_df=pd.concat([newclustered_df,clustered_df["Class"]], axis=1)
newclustered_df

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.200000e-11,0.000000e+00,42 Coin,0
2,5.320000e-04,1.065855e-03,404Coin,0
5,3.141593e-01,2.957551e-02,EliteCoin,0
7,2.100000e-05,1.810842e-05,Bitcoin,3
8,0.000000e+00,1.087731e-04,Ethereum,3
...,...,...,...,...
1238,2.000000e-03,2.020225e-03,ZEPHYR,0
1242,2.500000e-04,1.508199e-05,Gapcoin,0
1245,1.400223e-03,9.901351e-04,Beldex,3
1246,2.100000e-05,7.370282e-06,Horizen,3


In [179]:
# Create an hvplot scatter plot with x="TotalCoinsMined", y="TotalCoinSupply", and by="Class", and have it show the CoinName when you hover over the the data point.
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
newclustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)